In [16]:
#SPARK TANITMA
#işletim sistemi linux olduğu için
import os
#sparkı bulma kütüphanesi
import findspark
#jupyter notebooka sparkı nereye kurduğumuzu söylüyoruz
findspark.init("/usr/local/spark/spark-3.0.1-bin-hadoop2.7/")

In [18]:
#SPARK'a bağlanma
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [14]:
#veri çekmek için kütüphane
import urllib
#eğitim veriseti çekilir
f = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")

data_file = "./kddcup.data.gz"

In [20]:
#training dataset
data_file

'./kddcup.data.gz'

In [24]:
#eğitim verisi için RDD oluşturulur
#Spark'ın textFile fonksiyonu sıkıştırılmış dosyaları doğrudan işler
raw_data = sc.textFile(data_file)


In [22]:
#eğitim verisi RDD
raw_data

./kddcup.data.gz MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [25]:
#RDD ye yüklenen satır sayısını saydırma
#yaklaşık 5 milyon satır eğitim verisi
raw_data.count()

4898431

In [35]:
#Eğitim verisinden ilk 10 satır
raw_data.take(10)

['0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2,2,1.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,3,3,1.00,0.00,0.33,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.00,0.00,0.00,0.00,1.00,0.00,0.00,4,4,1.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,238,1282,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.00,0.00,0.00,0.00,1.00,0.00,0.00,5,5,1.00,0.00,0.20,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.00,0.00,0.00,0.00,1.00,0.00,0.00,6,6,

In [31]:
#test verisi çekilir
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz", "corrected.gz")
test_data_file = "./corrected.gz"
ft

('corrected.gz', <http.client.HTTPMessage at 0x7f639c49f0f0>)

In [28]:
#test verisinden RDD oluşturulur
test_raw_data = sc.textFile(test_data_file)

In [29]:
#test veri dosyası RDD
test_raw_data

./corrected.gz MapPartitionsRDD[10] at textFile at NativeMethodAccessorImpl.java:0

In [30]:
#RDD ye yüklenen satır sayısını saydırma
#300 bin satır
test_raw_data.count()

311029

In [34]:
#test verisinden ilk 10 satır
test_raw_data.take(10)

['0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,snmpgetattack.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,snmpgetattack.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,snmpgetattack.',
 '0,udp,domain_u,SF,29,0,0,0,0,0,0,0,0,0,0,

In [43]:
#MLlib makine öğrenmesi kütüphanesini ekleme
from pyspark.mllib.regression import LabeledPoint
from numpy import array

In [44]:
#satırdaki etiketi normalse saldırı değil, farklı ise saldırı olarak etiketleyen fonksiyon 
#normal:0 attack: 1 şeklinde sayısallaştırır. 
#İlk 4 öznitelik veriden çıkarılır. Veri önişleme gerçekleştiriip veri analize hazırlanır.
def parse_interaction(line):
    line_split = line.split(",")
    # öznitelik 1,2,3,4 dahil edilmez
    clean_line_split = line_split[0:1]+line_split[4:41]
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))


In [45]:
#preparing train data
training_data = raw_data.map(parse_interaction)


In [46]:
#preparing test data
test_data = test_raw_data.map(parse_interaction)

In [47]:
#binary logistic regression classification kütüphanesi ekleme
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

In [50]:
# training 
logit_model = LogisticRegressionWithLBFGS.train(training_data)

In [111]:
#prediction
prediction = test_data.map(lambda p: (p.label, logit_model.predict(p.features)))

In [112]:
prediction

PythonRDD[383] at RDD at PythonRDD.scala:53

In [126]:
from sklearn.metrics import accuracy_score
import numpy as np

In [127]:
predictionAndTargetNumpy = np.array((prediction.collect()))

acc = accuracy_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])

In [128]:
acc

0.8625819457349636